In [218]:
import utils
import importlib
import polars as pl

importlib.reload(utils)

DATA_PATH = "./data/"

LIMIT_DATA = 100000

social_graph = utils.SocialGraph(source_file=DATA_PATH + "social_graph.csv")
social_graph.build(limit=LIMIT_DATA)



dataset = utils.CandidateDataset(
    source_train=DATA_PATH + "train.csv",
    source_test=DATA_PATH + "test.csv",
    source_feature_data=DATA_PATH + "feature_metadata.json",
)


dataset.build_datasets(limit=None)
# dataset.display_feature_data()
dataset.df_train.head()


Successfully built social graph with 104023 nodes.
Imported train dataset with 272819 entries (272819 of which have missing features)
Imported test dataset with 48416 entries (15162 of which have missing features)


user_hash,feature_001,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_012,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
"""680f47f06ba8f132""",1.0,6.0,7.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.02919,1,0.133333,0.75,1.0,null
"""dbf485f686e509c6""",null,null,null,3.0,2.0,null,null,null,null,null,null,null,null,null,101.339549,1,5.466667,null,null,1.0
"""b49927482049a4ea""",1.0,2.0,9.0,1.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,37.091111,3,8.003991,0.7975,1.0,null
"""334d8c0aca4b2fd9""",3.0,7.0,6.0,2.0,8.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.049236,2,21.691667,0.75,1.0,null
"""363d12825b7b131a""",1.0,2.0,6.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048692,1,13.316667,0.61,1.0,null


In [219]:
dataset.apply_feature_typing()
dataset.df_train.head()

Applying feature types to datasets...


user_hash,feature_001,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_012,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating
str,cat,i8,i8,i8,i8,i8,bool,i8,i8,f32,bool,cat,bool,bool,f32,f32,f32,f32,bool,bool
"""680f47f06ba8f132""","""1""",6,7,1,1,1,true,0,0,0.0,false,"""0""",true,true,0.02919,1.0,0.133333,0.75,true,false
"""dbf485f686e509c6""","""missing""",1,1,3,2,0,false,0,0,809.05426,false,"""missing""",false,false,101.339546,1.0,5.466667,0.574001,false,true
"""b49927482049a4ea""","""1""",2,9,1,1,6,true,0,0,0.0,false,"""2""",true,true,37.09111,3.0,8.00399,0.7975,true,false
"""334d8c0aca4b2fd9""","""3""",7,6,2,8,6,true,0,0,0.0,false,"""0""",false,true,0.049236,2.0,21.691668,0.75,true,false
"""363d12825b7b131a""","""1""",2,6,1,4,0,true,0,0,0.0,false,"""0""",false,false,0.048692,1.0,13.316667,0.61,true,false


In [192]:
dataset.df_test.dtypes

[String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 Float64]

In [ ]:
dataset.df_train.filter(pl.col("user_hash") == "a79a4cd7a37b1fb2")

user_hash,feature_001,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_012,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
"""a79a4cd7a37b1fb2""",1.0,2.0,8.0,1.0,6.0,6.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,12.000521,2,16.408333,0.0,1.0,null
